# Worksheet 05

Name: Mahdi Khemakhem
UID: U18251472

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

Run 1:
Cluster 1 = [0, 0.5], mean = 0.25
Cluster 2 = [1.5, 2, 6, 6.5, 7] mean = 4.6

Run 2:
Cluster 1 = [0, .5, 1.5, 2] mean = 1.0
Cluster 2 = [6, 6.5, 7] mean = 6.5

Will stop after run 2.

b) Describe in plain english what the cost function for k means is.

Minimizing the distance function for each point.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

Different local minima for the optimization problem, which would depend on the initial random choice of centroids.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, because there are only so many ways to partition the data into K clusters, and with each step we either improve or we do not change because we have already found a good cluster.

e) Follow along in class the implementation of Kmeans

In [9]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))


    def initialize(self):
        return self.data[np.random.choice(range(len(self.dataset)), self.k)]

    def assign(self, centers):
        for i in range(len(self.data)):
            minimum = self.distance(self.data[i], centers[0])
            for j in range(len(self.centers)):
                dist = self.distance(self.data[i], centers[j])
                if dist < minimum:
                    minimum = dist
                    self.assignment[i] = j

    def get_centers(self):
        for i in set(self.assignment):
            cluster = []
            for j in range(len(self.data)):
                if self.assignment[j] == i:
                    cluster.append(self.data[j])
            centers.append(np.mean(cluster))
        return centers

    def distance(self, x, y):
        return np.linalg.norm(x - y)

    def is_diff_clusters(self, centers, new_centers):
        for i in range(len(centers)):
            if self.distance(centers[i], new_centers[i]) != 0:
                return True
        return False

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.get_centers()
        while self.is_diff_clusters(centers, new_centers):
            self.assign(new_centers)
            centers = new_centers
            self.snap(new_centers)
            new_centers = self.get_centers()
        return 


    def lloyds_personal(self):
        centers = self.data[np.random.choice(self.data.shape[0], self.k, replace=False)]

        while True:
            distances = np.linalg.norm(self.data[:, np.newaxis] - centers, axis=2)
            new_assignment = np.argmin(distances, axis=1)
            
            # Check for convergence
            if np.array_equal(new_assignment, self.assignment):
                break

            self.assignment = new_assignment

            # Update step: Calculate new centroids
            new_centers = np.array([self.data[self.assignment == i].mean(axis=0) for i in range(self.k)])
            
            self.snap(new_centers)  # Take a snapshot after updating centroids

            centers = new_centers
            

kmeans = KMeans(X, 6)
kmeans.lloyds_personal()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)